In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U
!pip install accelerate>=0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=ee2696917401270a82157eb97e95f4c7ee057364d591f5c02728fab973e7ceb3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.9 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


In [2]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [4]:
# let us try with some example sentences here
sentence = 'लगातार हमलावर हो रहे शिवपाल और राजभर को सपा की दो टूक, चिट्ठी जारी कर कहा- जहां जाना चाहें जा सकते हैं'

predicted_labels = get_predictions(sentence=sentence, 
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

लगातार	O
हमलावर	O
हो	O
रहे	O
शिवपाल	B-PER
और	O
राजभर	B-PER
को	O
सपा	B-ORG
की	O
दो	O
टूक,	O
चिट्ठी	O
जारी	O
कर	O
कहा-	O
जहां	O
जाना	O
चाहें	O
जा	O
सकते	O
हैं	O


In [5]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [7]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [8]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [9]:
# let's print an instance of dataset
idx=1000
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


सूक्ष्म	0
,	0
लघु	0
एवं	0
मध्यम	0
उद्यम	0
(	0
एमएसएमई	3
)	0
और	0
सड़क	3
परिवहन	4
एवं	4
राजमार्ग	4
मंत्री	0
श्री	0
नितिन	1
गडकरी	2
ने	0
आज	0
जानकारी	0
दी	0
कि	0
उनका	0
मंत्रालय	0
एक	0
कृषि	0
एमएसएमई	0
नीति	0
लाने	0
पर	0
काम	0
कर	0
रहा	0
है	0
जो	0
स्थानीय	0
कच्चे	0
माल	0
का	0
उपयोग	0
करते	0
हुए	0
विनिर्माण	0
उत्पादों	0
के	0
लिए	0
ग्रामीण	0
,	0
जनजातीय	0
,	0
कृषि	0
और	0
वन	0
क्षेत्रों	0
में	0
उद्यमिता	0
विकास	0
पर	0
ध्यान	0
केंद्रित	0
करेगी	0
।	0


In [10]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [11]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [12]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [13]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

2024-03-13 13:28:46.828611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 13:28:46.828717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 13:28:46.973896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

In [15]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [45]:
ot=len(raw_datasets["train"])
nts=int(ot*0.02)
train_dataset = raw_datasets["train"].select(range(nts))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [17]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [19]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [20]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    weight_decay=0.01  # Set the weight decay value here
)


In [21]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.323700,0.297312,0.726742,0.621854,0.670220,10213,0.479573,0.527795,0.502530,9786,0.685258,0.658024,0.671365,10568,0.622787,0.604246,0.613377,0.909288
2,0.258300,0.267385,0.718898,0.682366,0.700156,10213,0.550037,0.536378,0.543122,9786,0.677573,0.713285,0.694971,10568,0.650682,0.646318,0.648493,0.917162
3,0.208400,0.263925,0.704824,0.725252,0.714892,10213,0.573421,0.533517,0.552750,9786,0.700608,0.709027,0.704792,10568,0.663862,0.658259,0.661049,0.919292


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [23]:
model.save_pretrained('my_indicBERT')

In [24]:
tokenizer.save_pretrained("tokenizer_indicBERT")

('tokenizer_indicBERT/tokenizer_config.json',
 'tokenizer_indicBERT/special_tokens_map.json',
 'tokenizer_indicBERT/spiece.model',
 'tokenizer_indicBERT/added_tokens.json',
 'tokenizer_indicBERT/tokenizer.json')

In [25]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [26]:
import json

In [27]:
config = json.load(open("/kaggle/working/my_indicBERT/config.json"))

In [28]:
config["id2label"]=id2label
config["label2id"]=label2id

In [29]:
json.dump(config, open("/kaggle/working/my_indicBERT/config.json","w"))


In [30]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/my_indicBERT')

In [31]:
model

AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(200000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, b

In [32]:
import torch

def get_ner_from_file(file_path):
    with open(file_path, 'r') as file:
        sentences = file.readlines()
    return get_ner(sentences)

def get_ner(sentences):
    ner_outputs = []
    
    for sentence in sentences:
        tok_sentence = tokenizer(sentence, return_tensors='pt')

        with torch.no_grad():
            logits = model(**tok_sentence).logits.argmax(-1)
            predicted_tokens_classes = [
                model.config.id2label[t.item()] for t in logits[0]]

            predicted_labels = []

            previous_token_id = 0
            word_ids = tok_sentence.word_ids()
            for word_index in range(len(word_ids)):
                if word_ids[word_index] == None:
                    previous_token_id = word_ids[word_index]
                elif word_ids[word_index] == previous_token_id:
                    previous_token_id = word_ids[word_index]
                else:
                    predicted_labels.append(predicted_tokens_classes[word_index])
                    previous_token_id = word_ids[word_index]

            ner_output = []
            for index in range(len(sentence.split(' '))):
                if(index < len(predicted_labels)):
                    ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
                else:
                    ner_output.append((sentence.split(' ')[index], 'O'))
            ner_outputs.append(ner_output)
    
    return ner_outputs

# Example usage
file_path = "/kaggle/input/aditya/katare.txt"
ner_results = get_ner_from_file(file_path)
print(ner_results)


[[('\ufeffलगातार', 'O'), ('किसान', 'O'), ('इस', 'O'), ('बाबत', 'O'), ('जेडीए', 'O'), ('में', 'O'), ('आकर', 'O'), ('योजना', 'O'), ('को', 'O'), ('पूर्ण', 'O'), ('करने', 'O'), ('के', 'O'), ('लिये', 'O'), ('मांग', 'O'), ('कर', 'O'), ('रहे', 'O'), ('है।\n', 'O')], [('राजे', 'O'), ('छह', 'O'), ('राज्यों', 'O'), ('के', 'O'), ('मुख्यमंत्रियों', 'O'), ('के', 'O'), ('साथ', 'O'), ('नीति', 'O'), ('आयोग', 'I-ORG'), ('की', 'O'), ('बैठक', 'O'), ('में', 'O'), ('भाग', 'O'), ('लेने', 'O'), ('दिल्ली', 'B-LOC'), ('आईं', 'O'), ('थी।\n', 'O')], [('राजीव', 'B-PER'), ('गाँधी', 'I-PER'), ('ने', 'O'), ('प्रधान', 'O'), ('मंत्री', 'O'), ('कार्यकाल', 'O'), ('के', 'O'), ('दौरान', 'O'), ('उन्हें', 'O'), ('उत्कृष्ट', 'O'), ('कार्यों', 'O'), ('के', 'O'), ('लिए', 'O'), ('पुरस्कृत', 'O'), ('भी', 'O'), ('किया', 'O'), ('था.\n', 'O')], [('कई', 'O'), ('अप्रिय', 'O'), ('घटनाओं', 'O'), ('से', 'O'), ('आपका', 'O'), ('मन', 'O'), ('खिन्न', 'O'), ('रह', 'O'), ('सकता', 'O'), ('है।', 'O'), ('\n', 'O')], [('यही', 'O'), ('पानी', 'O'),

In [33]:
!ls

'=0.21.0'   my_indicBERT   output_dir   tokenizer_indicBERT   wandb


In [34]:
!zip -r BertTuned.zip /kaggle/working/my_indicBERT

  adding: kaggle/working/my_indicBERT/ (stored 0%)
  adding: kaggle/working/my_indicBERT/model.safetensors (deflated 7%)
  adding: kaggle/working/my_indicBERT/config.json (deflated 51%)


In [35]:
!zip -r BertTuned_tokenizer.zip /kaggle/working/tokenizer_indicBERT

  adding: kaggle/working/tokenizer_indicBERT/ (stored 0%)
  adding: kaggle/working/tokenizer_indicBERT/tokenizer_config.json (deflated 74%)
  adding: kaggle/working/tokenizer_indicBERT/special_tokens_map.json (deflated 49%)
  adding: kaggle/working/tokenizer_indicBERT/spiece.model (deflated 60%)
  adding: kaggle/working/tokenizer_indicBERT/tokenizer.json (deflated 77%)


In [36]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

  0%|          | 0/3 [00:00<?, ?it/s]

In [37]:
test_dataset = raw_datasets["test"]
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [46]:
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [39]:
print(metrics)

{'test_loss': 0.21802595257759094, 'test_LOC_precision': 0.6930860033726813, 'test_LOC_recall': 0.6693811074918566, 'test_LOC_f1': 0.6810273405136702, 'test_LOC_number': 614, 'test_ORG_precision': 0.6236363636363637, 'test_ORG_recall': 0.6533333333333333, 'test_ORG_f1': 0.638139534883721, 'test_ORG_number': 525, 'test_PER_precision': 0.7237977805178791, 'test_PER_recall': 0.7430379746835443, 'test_PER_f1': 0.7332916926920675, 'test_PER_number': 790, 'test_overall_precision': 0.6862845445240532, 'test_overall_recall': 0.6951788491446346, 'test_overall_f1': 0.6907030646407417, 'test_overall_accuracy': 0.9324385462222893, 'test_runtime': 16.4676, 'test_samples_per_second': 52.649, 'test_steps_per_second': 3.34}


In [49]:
predictions, labels, metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [51]:
print(metrics)

{'test_loss': 0.18889659643173218, 'test_LOC_precision': 0.7713865293291419, 'test_LOC_recall': 0.7906354973606636, 'test_LOC_f1': 0.7808924097772361, 'test_LOC_number': 14587, 'test_ORG_precision': 0.6738220706757595, 'test_ORG_recall': 0.625, 'test_ORG_f1': 0.6484934367541767, 'test_ORG_number': 13912, 'test_PER_precision': 0.7972374250716706, 'test_PER_recall': 0.7944422802233476, 'test_PER_f1': 0.7958373983739837, 'test_PER_number': 15402, 'test_overall_precision': 0.7514292988912807, 'test_overall_recall': 0.7394820163549806, 'test_overall_f1': 0.7454077883908891, 'test_overall_accuracy': 0.9434812917009932, 'test_runtime': 375.1096, 'test_samples_per_second': 52.558, 'test_steps_per_second': 3.287}
